In [0]:
#create data

person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])
]).toDF("id","name","graduate_program","spark_status")

graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
(500, "Vice President"),
(250, "PMC Member"),
(100, "Contributor")])\
.toDF("id", "status")

person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [0]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression).show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#118L], [id#132L], Inner
 :- Sort [graduate_program#118L ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(graduate_program#118L, 200), true, [id=#2931]
 : +- Project [_1#108L AS id#116L, _2#109 AS name#117, _3#110L AS graduate_program#118L, _4#111 AS spark_status#119]
 : +- Filter isnotnull(_3#110L)
 : +- Scan ExistingRDD[_1#108L,_2#109,_3#110L,_4#111]
 +- Sort [id#132L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#132L, 200), true, [id=#2932]
 +- Project [_1#124L AS id#132L, _2#125 AS degree#133, _3#126 AS department#134, _4#127 AS school#135]
 +- Filter isnotnull(_1#124L)
 +- Scan ExistingRDD[_1#124L,_2#125,_3#126,_4#127]

In [0]:
person.join(graduateProgram, joinExpression, "outer").show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [0]:
graduateProgram.join(person, joinExpression, "left_outer").show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 2|Masters| EECS|UC Berkeley|null| null| null| null|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+

In [0]:
person.join(graduateProgram, joinExpression, "right_outer").show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [0]:
#nothing from right data frame is included, behaves like a filter
graduateProgram.join(person, joinExpression, "left_semi").show()

+---+-------+--------------------+-----------+
 id| degree| department| school|
+---+-------+--------------------+-----------+
 0|Masters|School of Informa...|UC Berkeley|
 1| Ph.D.| EECS|UC Berkeley|
+---+-------+--------------------+-----------+

In [0]:
#opp of semi joins. Keeps only values that do not exist in the 2nd df.
graduateProgram.join(person,joinExpression, "left_anti").show()

+---+-------+----------+-----------+
 id| degree|department| school|
+---+-------+----------+-----------+
 2|Masters| EECS|UC Berkeley|
+---+-------+----------+-----------+

In [0]:
person.join(graduateProgram, joinExpression, "cross").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

### Challenges of Joins ###

In [0]:
from pyspark.sql.functions import expr,col
person.withColumnRenamed("id","personId")\
  .join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
personId| name|graduate_program| spark_status| id| status|
+--------+----------------+----------------+---------------+---+--------------+
 0| Bill Chambers| 0| [100]|100| Contributor|
 1| Matei Zaharia| 1|[500, 250, 100]|500|Vice President|
 1| Matei Zaharia| 1|[500, 250, 100]|250| PMC Member|
 1| Matei Zaharia| 1|[500, 250, 100]|100| Contributor|
 2|Michael Armbrust| 1| [250, 100]|250| PMC Member|
 2|Michael Armbrust| 1| [250, 100]|100| Contributor|
+--------+----------------+----------------+---------------+---+--------------+

In [0]:
gradProgramDupe = graduateProgram.withColumnRenamed("id","graduate_program")
#now person and gradProgramDupe have column with same name
joinExp = person["graduate_program"]==gradProgramDupe["graduate_program"]

## error
#person.join(gradProgramDupe, joinExp).select("graduate_program").show()

#1 Change join Expression from boolean to string
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show()


#2 Drop the column after join
person.join(gradProgramDupe, joinExp).drop(person["graduate_program"]).select("graduate_program").show()

#3 Rename column before join
gradProgram3 = graduateProgram.withColumnRenamed("id","grad_id")
jexp = person["graduate_program"] == gradProgram3["grad_id"]
person.join(gradProgram3, jexp).select("graduate_program").show()

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

In [0]:
person.join(graduateProgram, joinExpression).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#118L], [id#132L], Inner
 :- Sort [graduate_program#118L ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(graduate_program#118L, 200), true, [id=#2844]
 : +- Project [_1#108L AS id#116L, _2#109 AS name#117, _3#110L AS graduate_program#118L, _4#111 AS spark_status#119]
 : +- Filter isnotnull(_3#110L)
 : +- Scan ExistingRDD[_1#108L,_2#109,_3#110L,_4#111]
 +- Sort [id#132L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#132L, 200), true, [id=#2845]
 +- Project [_1#124L AS id#132L, _2#125 AS degree#133, _3#126 AS department#134, _4#127 AS school#135]
 +- Filter isnotnull(_1#124L)
 +- Scan ExistingRDD[_1#124L,_2#125,_3#126,_4#127]

In [0]:
#hint to the optimizer to use broadcast join
from pyspark.sql.functions import broadcast
person.join(broadcast(graduateProgram), joinExpression).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [graduate_program#118L], [id#132L], Inner, BuildRight
 :- Project [_1#108L AS id#116L, _2#109 AS name#117, _3#110L AS graduate_program#118L, _4#111 AS spark_status#119]
 : +- Filter isnotnull(_3#110L)
 : +- Scan ExistingRDD[_1#108L,_2#109,_3#110L,_4#111]
 +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true])), [id=#3017]
 +- Project [_1#124L AS id#132L, _2#125 AS degree#133, _3#126 AS department#134, _4#127 AS school#135]
 +- Filter isnotnull(_1#124L)
 +- Scan ExistingRDD[_1#124L,_2#125,_3#126,_4#127]